Modelo de Dou et al. (2020)

Agora em Julia versão 1.5.3

In [1]:
# using Pkg

# Pkg.add("ProfileVega")

In [2]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters

using BenchmarkTools

#packages to increase the speed of the code
using Profile # @profile command
using ProfileVega #profiling using graphs
using Traceur #analyses if your function has performance flaws such as changing the type of a variable


Parâmetros do Jogo

In [3]:
const μ = Float64(4.566) #número de meses entre períodos, traduz a quantidade de períodos t em quantidade de meses
const ρ = Float64(0.884) # (1 - ρ) é a taxa de depreciação da empresa a cada período
# const ρ = Float64(0.5) #testando nova taxa de depreciação
const β = Float64(9.84) #inverso da velocidade de aprendizado
const c0 = Float64(0.044) #custo fixo de ir para a corte
const c1 = Float64(0.015) #custo variável de ir para a corte


const hs0 = Float64(0.28) #habilidade inicial de s
const hj0 = Float64(0.36) #habilidade inicial de j

const λj = Float64(0.346)

#const is to alert Julia that these global variables will not change, so she does not have to worry about then

0.346

Valores que virão dos dados

In [4]:
const Vmax = Float64(1.0) #valor máximo de reorganização da firma
const L = Float64(0.25) #valor inicial de liquidação da firma
const Ds = Float64(0.32) #valor da dívida com credor sênior
const Dj = Float64(0.68) #valor da dívida com credor júnior

const D = Ds + Dj #valor total da dívida, usada para escalar custos

1.0

Definições iniciais: custos, valor máximo da firma, número de rounds

In [5]:
#função custo
function Ct(t)
    #cost at period t=0(index1) is 0
    if(t == 1)
        return 0
    else
        return c0 * D + c1 * (t-1) * D
    end
end

#precisa de um 'end' para o if e um 'end' para a função
    
#test
Ct(0)    
Ct(1)
Ct(2)
Ct(2) == c0 * D + c1 * (1) * D

#o fato de o custo ser função de D implica que duas empresas com mesmo valor de V e L podem ter destinos diferentes, pois
#a empresa com maior D terá maiores custos dentro da corte

true

In [6]:
#number of periods
function max_turns(Vmax, L, ρ)
    t = 0
    
    while(ρ^(t-1) * Vmax > L)
        t = t+1
    end
    return t
end


T = max_turns(Vmax, L, ρ)
T

13

In [7]:
#testando se T é o período final mesmo
@show t = T
@show ρ^(t-1) * Vmax - Ct(t) > L - Ct(t)


@show t = T-1
@show ρ^(t-1) * Vmax - Ct(t) > L - Ct(t)


#to see the values
@show t = T-1
@show ρ^(t-1) * Vmax - Ct(t)

@show L - Ct(t)

t = T = 13
ρ ^ (t - 1) * Vmax - Ct(t) > L - Ct(t) = false
t = T - 1 = 12
ρ ^ (t - 1) * Vmax - Ct(t) > L - Ct(t) = true
t = T - 1 = 12
ρ ^ (t - 1) * Vmax - Ct(t) = 0.04861723529489925
L - Ct(t) = 0.041000000000000036


0.041000000000000036

In [34]:
#função com valor máximo de reorganização da firma a cada período
function Vt(Vmax, Tmax, ρ)
    
    V = Array{Float64}(undef, Tmax+1)
    V[1] = Vmax
    
    #range exclui o valor máximo, daí que eu adicionei 1
    
    for t in 2:Tmax+1
        
        V[t] = Vmax * ρ^(t-1)
    end
    
    return V
end

V = Vt(Vmax, T, ρ)
V


14-element Array{Float64,1}:
 1.0
 0.884
 0.781456
 0.690807104
 0.6106734799360001
 0.539835356263424
 0.4772144549368668
 0.4218575781641903
 0.37292209909714424
 0.3296631356018755
 0.29142221187205797
 0.2576172352948992
 0.2277336360006909
 0.20131653422461077

In [35]:
#array with costs each period

C = similar(V)

C[1] = 0

for t in 2:T+1
    C[t] = Ct(t)
end
    
C

14-element Array{Float64,1}:
 0.0
 0.059
 0.074
 0.089
 0.104
 0.119
 0.134
 0.149
 0.16399999999999998
 0.179
 0.194
 0.20899999999999996
 0.22399999999999998
 0.239

In [36]:
#liquidation payoffs

function s_L(t)
    return min(L - Ct(t), Ds)
end


@show s_L(1)
@show s_L(2)


function j_L(t)
    return min(L - Ct(t) - s_L(t), Dj)
end
    
@show j_L(1)

s_L(1) = 0.25
s_L(2) = 0.191
j_L(1) = 0.0


0.0

Skill levels' grid and Probability Mass Function(PMF)

Important: we won't use the skill levels directly in the functions, we will use their indexes. For example, index = 40 corresponds to $\theta_{K,t} = 0.4$.

In [37]:
#grid size
grid = 100

hlow = 0.01
hhigh = 1.0

#começa no menor valor possível, vai até o maior valor possível num intervalo do tamanho do grid
hvals = LinRange(hlow, hhigh, grid)

hvals
hvals[6]

0.06

In [38]:
#array to tell us the size of the pie at period t
#arguments: (period, θk,t)
U = Array{Float64}(undef,T+1, grid)


#cartesianIndices returns the index(i,j), and index(i,j)[1] = i, index(i,j)[2] = j
for i in CartesianIndices(U)
    U[i] = hvals[i[2]] * V[i[1]] - C[i[1]]
end  

#if t==0 (index 1), then there are no costs
@show U[1,7] == hvals[7] * V[1]

@show U[1, 7] == hvals[7] * Vmax

#if t>0 (index>1), there are costs
@show U[2,1] == hvals[1] * V[2] - Ct(2)

#checking the last period
#t = T
@show U[T,7] ==  hvals[7] * V[T] - Ct(T)

U[1, 7] == hvals[7] * V[1] = true
U[1, 7] == hvals[7] * Vmax = true
U[2, 1] == hvals[1] * V[2] - Ct(2) = true
U[T, 7] == hvals[7] * V[T] - Ct(T) = true

true

In [39]:
#cumulative distribution function
function cdf(x, lt)
    
    #return the cdf of x given the lower bound lt
    #geq than 1 because of our discretization method 
    if(x >= 1)
        
        return 1
    
    else
        
        if(x >= lt)
            return 1 - ((1-x)^β)/((1-lt)^β)
        else
            return 0
        end
    end
end

#test      
@show cdf(0.5, 1)

@show cdf(0.5, 0.4)

@show cdf(0.99, 0.01)

@show cdf(1, 0.01)

@show cdf(1, 0.9)

cdf(0.5, 1) = 0
cdf(0.5, 0.4) = 0.8337136736331454
cdf(0.99, 0.01) = 1.0
cdf(1, 0.01) = 1
cdf(1, 0.9) = 1


1

In [40]:
#nova versão da função pmf: agora cada ponto do grid é o centro da cdf
#pmf: each point of the grid is at the center of the cdf
function pmf_cdf(lt, δ = hlow/2)
    pmf = zeros(grid)
    
    for (i, h) in enumerate(hvals)
        pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
    end
    
    return pmf
end


lt = 0.01
pmf_cdf(lt)

100-element Array{Float64,1}:
 0.048602089527521986
 0.09088953125821697
 0.08301186884522238
 0.07574579901540057
 0.06904947823155516
 0.06288353216214904
 0.05721093367839869
 0.05199688568380789
 0.04720870863310611
 0.04281573260029048
 0.038789193758253626
 0.03510213513518168
 0.03172931151561542
 ⋮
 3.7337122282821156e-10
 1.615731992643532e-10
 6.408462649432067e-11
 2.2833512858255745e-11
 7.107758825952715e-12
 1.856736986383112e-12
 3.83026943495679e-13
 5.651035195342047e-14
 4.9960036108132044e-15
 2.220446049250313e-16
 0.0
 0.0

In [41]:
#generating the pmfs

#probability mass function
pmf = Array{Float64}(undef,grid,grid)


for (t, θt) in enumerate(hvals)
    pmf[t,:] = pmf_cdf(θt)
end


#we need a pmf for when we have lkt and want to know θk,t+1
#we will call it 'pmf2'
pmf2 = similar(pmf)

for i in eachindex(hvals)
    for j in eachindex(hvals)
        pmf2[i, j] = sum(pmf[i, :] .*pmf[:, j])
    end
end

#rows are the skill levels today, columns are the skill levels tomorrow
pmf2[1,:] - pmf[1,:]

#should be nice to plot the two pmfs, so we will get the intuition easily

100-element Array{Float64,1}:
 -0.046239926421080726
 -0.08201062544335427
 -0.06652389304197329
 -0.05297653770265348
 -0.04117778693388258
 -0.03095216055945394
 -0.022138473358402036
 -0.014588889342129194
 -0.008168025651978328
 -0.0027521041142639507
  0.0017718514572064026
  0.0055067740688657385
  0.008546270271650315
  ⋮
  7.710965259700306e-9
  3.4895426000488317e-9
  1.4511143376026725e-9
  5.43807346649936e-10
  1.7875565752175066e-10
  4.95698020399729e-11
  1.093125330283789e-11
  1.7426855209729608e-12
  1.6978822815991093e-13
  7.239768851825965e-15
  5.665150746454088e-17
  8.758153619997799e-22

Arrays with the continuation values

In [42]:
#continuation values
#period t (not t+1), θkt, ℓkt, ℓmt
s_W = zeros(T, grid, grid, grid)
j_W = copy(s_W)


#optimal payments
#period, θkt, ℓmt, outputs
Pst_array = zeros(T, grid, grid, 6)
Pjt_array = copy(Pst_array)

13×100×100×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.

In [43]:
#populating the last period with the liquidation values
s_W[T, :, :, :] .= s_L(T) #we need to use .= because it is a broadcast
j_W[T, :, :, :] .= j_L(T)

100×100×100 view(::Array{Float64,4}, 13, :, :, :) with eltype Float64:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  

In [44]:
#cutoff function: upgrade

function upcutoff_m(t, Pkt, hkt, m_W)

    #we extract the diagonal values, i.e., where hmt == lmt, to search for the cutoff
    diag_vals = @views diag(m_W[t+1, :, :, hkt])
    
    return searchsortedfirst(diag_vals, Pkt)
end

#should return 100 because J's continuation value at T is zero
@btime upcutoff_m(T-1, 0.01, 50, j_W)


  240.098 ns (1 allocation: 896 bytes)


101

In [45]:
#function to update the lowerbound according to the cutoff

function cutoff_m(t, Pkt, hkt, lmt, m_W)
    cmt = upcutoff_m(t, Pkt, hkt, m_W)
    
    if(cmt==101)
        return lmt
    else
        return max(cmt, lmt)
    end
end


@btime cutoff_m(T-1, 0.01, 50, 40, j_W)

  237.634 ns (1 allocation: 896 bytes)


40

In [46]:
#function to select the pmf function as it is writen in the paper

function h_m(t, lmt)
    #takes the indexes of cutoff and lmt
        
    #at t=0(index 1), all information is symmetric, so we use pmf instead of pmf2mm
    if(t <= 1)
        return pmf[lmt, lmt:end]
    else
        return pmf2[lmt, lmt:end]
    end
end
    
#test
@show h_m(1, 50) == pmf[50, 50:end]

@show h_m(2, 50) == pmf2[50, 50:end]

@btime h_m(2,50)

h_m(1, 50) == pmf[50, 50:end] = true
h_m(2, 50) == pmf2[50, 50:end] = true
  136.828 ns (2 allocations: 528 bytes)


51-element Array{Float64,1}:
 0.008866568891510005
 0.03134054034293772
 0.053998301693260115
 0.06846396707993126
 0.07657685858255744
 0.07985914015113041
 0.0795594757138284
 0.07669193629092115
 0.07207054732437024
 0.06633984527073677
 0.06000179102641623
 0.05343936695638518
 0.046937164177679076
 ⋮
 4.625716199215591e-6
 2.128501871030038e-6
 8.99899204442307e-7
 3.428623147305904e-7
 1.1459581460290833e-7
 3.2321165203169804e-8
 7.253369111549412e-9
 1.1778647340830674e-9
 1.1710929608790348e-10
 5.110233142412765e-12
 4.3007226426965164e-14
 1.7849057398291997e-18

In [51]:
share = [0.0:hlow:hhigh;] #possible shares of the pie


#function to calculate the optimal payment offer
function Pkt(t, hkt, lmt, k_W, m_W, k_L)

    Pkt_grid = zeros(length(share) + 1)

    #the first possible value is -Vmax, which is an offer so low that will be rejected for sure (waiting offer)
    Pkt_grid[1] = -Vmax

    #the other values are the possible shares of the pie today
    Pkt_grid[2:end] = U[t, hkt] * share

    #descartando os valores que são menores que o menor valor de continuação possível do adversário
    #assim não confundiremos uma proposta de reorganização com uma de espera
    lowest = m_W[t+1, lmt, lmt, hkt]

    Pkt_grid[2:end] = @. ifelse(Pkt_grid[2:end] < lowest, lowest, Pkt_grid[2:end])

    cutoffs = Array{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cutoffs
    for (i, Pkt) in enumerate(Pkt_grid)
        cutoffs[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
    end

    #array com as probabilidades

    #probabilidade de m amanhã, dado o lower bound hoje ou o cutoff hoje

    #shape is Pkt x M
    probm = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    for i in eachindex(Pkt_grid)
        probm[i, :] = h_m(t, lmt)
    end

    #probabiilidades de k amanhã dado a habilidade verdadeira hoje. 
    probk = pmf[hkt, hkt:end]

    #payoff if the proposal is accepted
    #pE1[3,6] is the third possible payment combined with the sixth hability above hkt
    # pE1 = np.array(U[t, None, find(hkt):] - Pkt_grid[:, None]) #python code

    pE1 = repeat(transpose(U[t, hkt:end]), length(Pkt_grid),1) .- Pkt_grid

    # pE1[10,50]

    #multiplicando pE1 pelas probabilides das habilidades futuras de k
    pE1 = pE1 .* repeat(transpose(probk), size(pE1)[1], 1)

    pE1 = sum(pE1, dims=2) #dims=2 is the dimension over which it will sum

    #ponderação pelas probabilidades de m
    # pE1 = np.multiply(pE1[:, None], probm)

    pE1 = repeat(pE1, 1, size(probm)[2]) .* probm

    #indicator array
    cont_vals = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    # cont_vals = Array{Float64}(undef, length(Pkt_grid), length(hvals[find(lmt):end]))

    #continuation values separated before and after screening cutoffs

    #if cutoffs > lmt, then we use lmt+1 = lmt to all theta below cutoffs and lmt+1 = cutoffs to all theta above cutoffs
    #in julia, range(a:b) includes b. it's different from python
    for i in eachindex(Pkt_grid)
        if(cutoffs[i] > lmt)#there is a lowerbound update
            cont_vals[i, 1:(cutoffs[i]-lmt)] .= m_W[t+1, lmt:(cutoffs[i]-1), lmt, hkt]
            cont_vals[i, (cutoffs[i]-lmt+1):end] .= m_W[t+1, cutoffs[i]:end, cutoffs[i], hkt]
        else
            cont_vals[i, 1:end] .= m_W[t+1, lmt:end, lmt, hkt]
        end
    end

    #array indicating if the possible continuation values of the adversary are leq than each payment offer

    # IE1 = np.where(cont_vals <= Pkt_grid[:,None] , 1, 0) #python code
    IE1 = @. ifelse(cont_vals <= Pkt_grid, 1,0)

    #payoff E1
    E1 = pE1 .* IE1

    #we sum along the columns (dims=2) so as to each row contain the expected payoff of proposing a payment offer
    E1 = sum(E1, dims=2)

    # #cálculo de E2####

    #E2 payoff can use the cutoffs directly, i.e., doesn't need to separate before and after cutoff
    #this is because pE2 will only happen if M declines the offer, which only happens if his ability is above the cutoff

    pE2 = k_W[t+1, hkt:end, hkt, cutoffs]

    #multiplying pE2 by the probabilities
    #multiplicando pE2 pelo array de probabilidades
    # pE2 = np.multiply(pE2, probk[None, :])
    # pE2 = np.sum(pE2, axis = 1)

    pE2 = pE2 .* repeat(probk,1 , size(pE2)[2])

    pE2 = sum(pE2, dims=1) #dims=2 is the dimension over which it will sum

    #pE2 = np.multiply(pE2[:,None], probm) #python code

    pE2 = repeat(transpose(pE2), 1, size(probm)[2]) .* probm

    #indicator array
    # IE2 = np.where(IE1 == 1, 0, 1) #python code


    IE2 = @. ifelse(IE1 ==1, 0,1)

    #payoff

    E2 = pE2 .* IE2

    E2 = sum(E2, dims=2)

    #mpayoffs matrix####
    matrix_payoff = vec(E1 + E2) #vec is to convert it to a single column-matrix

    payoff_reorg, index_reorg = findmax(matrix_payoff)

    payment = Pkt_grid[index_reorg]

    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = matrix_payoff[1]

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

    #returns the expected payoff of a waiting proposal to make debugging easier
    return payment, cutoffs[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy
    
end


#testing hk = 0.5
t, hkt, lmt, k_W, m_W, k_L = T-1, 50, 60, s_W, j_W, s_L
@show Pkt(t, hkt, lmt, k_W, m_W, k_L)

#testing hk = 1
t, hkt, lmt, k_W, m_W, k_L = T-1, 100, 60, s_W, j_W, s_L
@show Pkt(t, hkt, lmt, k_W, m_W, k_L)


@btime Pkt(t, hkt, lmt, k_W, m_W, k_L)

Pkt(t, hkt, lmt, k_W, m_W, k_L) = (-1.0, 60, 0.026000000000000075, 0.026000000000000075, 0.041000000000000036, 1)
Pkt(t, hkt, lmt, k_W, m_W, k_L) = (0.0, 60, 0.048617235294899315, 0.02600000000000005, 0.048617235294899315, 3)
  244.001 μs (1962 allocations: 573.97 KiB)


(0.0, 60, 0.048617235294899315, 0.02600000000000005, 0.048617235294899315, 3)

In [53]:
share = [0.0:hlow:hhigh;] #possible shares of the pie

#function to calculate the optimal payment offer
function Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)

    Pkt_grid = zeros(length(share) + 1)

    #the first possible value is -Vmax, which is an offer so low that will be rejected for sure (waiting offer)
    Pkt_grid[1] = -Vmax

    #the other values are the possible shares of the pie today
    Pkt_grid[2:end] = U[t, hkt] * share

    #descartando os valores que são menores que o menor valor de continuação possível do adversário
    #assim não confundiremos uma proposta de reorganização com uma de espera
    lowest = m_W[t+1, lmt, lmt, hkt]

    Pkt_grid[2:end] = @. ifelse(Pkt_grid[2:end] < lowest, lowest, Pkt_grid[2:end])

    cutoffs = Array{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cutoffs
    for (i, Pkt) in enumerate(Pkt_grid)
        cutoffs[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
    end

    #array com as probabilidades

    #probabilidade de m amanhã, dado o lower bound hoje ou o cutoff hoje

    #shape is Pkt x M
    probm = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    for i in eachindex(Pkt_grid)
        probm[i, :] = h_m(t, lmt)
    end

    #probabiilidades de k amanhã dado a habilidade verdadeira hoje. 
    probk = pmf[hkt, hkt:end]

    #payoff if the proposal is accepted
    #pE1[3,6] is the third possible payment combined with the sixth hability above hkt
    # pE1 = np.array(U[t, None, find(hkt):] - Pkt_grid[:, None]) #python code

    pE1 = repeat(transpose(U[t, hkt:end]), length(Pkt_grid),1) .- Pkt_grid

    # pE1[10,50]

    #multiplicando pE1 pelas probabilides das habilidades futuras de k
    pE1 = pE1 .* repeat(transpose(probk), size(pE1)[1], 1)

    pE1 = sum(pE1, dims=2) #dims=2 is the dimension over which it will sum

    #ponderação pelas probabilidades de m
    # pE1 = np.multiply(pE1[:, None], probm)

    pE1 = repeat(pE1, 1, size(probm)[2]) .* probm

    #indicator array
    cont_vals = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    # cont_vals = Array{Float64}(undef, length(Pkt_grid), length(hvals[find(lmt):end]))

    #continuation values separated before and after screening cutoffs

    #if cutoffs > lmt, then we use lmt+1 = lmt to all theta below cutoffs and lmt+1 = cutoffs to all theta above cutoffs
    #in julia, range(a:b) includes b. it's different from python
    for i in eachindex(Pkt_grid)
        if(cutoffs[i] > lmt)#there is a lowerbound update
            cont_vals[i, 1:(cutoffs[i]-lmt)] .= m_W[t+1, lmt:(cutoffs[i]-1), lmt, hkt]
            cont_vals[i, (cutoffs[i]-lmt+1):end] .= m_W[t+1, cutoffs[i]:end, cutoffs[i], hkt]
        else
            cont_vals[i, 1:end] .= m_W[t+1, lmt:end, lmt, hkt]
        end
    end

    #array indicating if the possible continuation values of the adversary are leq than each payment offer

    # IE1 = np.where(cont_vals <= Pkt_grid[:,None] , 1, 0) #python code
    IE1 = @. ifelse(cont_vals <= Pkt_grid, 1,0)

    #payoff E1
    E1 = pE1 .* IE1

    #we sum along the columns (dims=2) so as to each row contain the expected payoff of proposing a payment offer
    E1 = sum(E1, dims=2)

    # #cálculo de E2####

    #E2 payoff can use the cutoffs directly, i.e., doesn't need to separate before and after cutoff
    #this is because pE2 will only happen if M declines the offer, which only happens if his ability is above the cutoff

    pE2 = k_W[t+1, hkt:end, hkt, cutoffs]

    #multiplying pE2 by the probabilities
    #multiplicando pE2 pelo array de probabilidades
    # pE2 = np.multiply(pE2, probk[None, :])
    # pE2 = np.sum(pE2, axis = 1)

    pE2 = pE2 .* repeat(probk,1 , size(pE2)[2])

    pE2 = sum(pE2, dims=1) #dims=2 is the dimension over which it will sum

    #pE2 = np.multiply(pE2[:,None], probm) #python code

    pE2 = repeat(transpose(pE2), 1, size(probm)[2]) .* probm

    #indicator array
    # IE2 = np.where(IE1 == 1, 0, 1) #python code


    IE2 = @. ifelse(IE1 ==1, 0,1)

    #payoff

    E2 = pE2 .* IE2

    E2 = sum(E2, dims=2)

    #mpayoffs matrix####
    matrix_payoff = vec(E1 + E2) #vec is to convert it to a single column-matrix

    payoff_reorg, index_reorg = findmax(matrix_payoff)

    payment = Pkt_grid[index_reorg]

    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = matrix_payoff[1]

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

    #returns the expected payoff of a waiting proposal to make debugging easier
    Pkt_array[t, hkt, lmt, :] .= payment, cutoffs[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy
    
end

#testing hk = 0.5
t, hkt, lmt, k_W, m_W, k_L, Pkt_array = T-1, 50, 60, s_W, j_W, s_L, Pst_array
@show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)

#testing hk = 1
t, hkt, lmt, k_W, m_W, k_L = T-1, 100, 60, s_W, j_W, s_L, Pst_array
@show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)


@btime Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)
#cutoff is Float64 in the output, this can be improved

Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array) = [-1.0, 60.0, 0.026000000000000075, 0.026000000000000075, 0.041000000000000036, 1.0]
Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array) = [0.0, 60.0, 0.048617235294899315, 0.02600000000000005, 0.048617235294899315, 3.0]
  245.600 μs (1975 allocations: 574.59 KiB)


6-element view(::Array{Float64,4}, 12, 100, 60, :) with eltype Float64:
  0.0
 60.0
  0.048617235294899315
  0.02600000000000005
  0.048617235294899315
  3.0

In [23]:
function Pst(t, θst, ℓjt)
    return Pkt(t, θst, ℓjt, s_W, j_W, s_L, Pst_array)
end


function Pjt(t, θjt, ℓst)
    return Pkt(t, θjt, ℓst, j_W, s_W, j_L, Pjt_array)
end

Pjt (generic function with 1 method)

In [24]:
#populating the arrays to continue the tests
t = T-1
for h in 1:grid
    for l in 1:grid
            Pst(t, h, l)
            Pjt(t, h, l)
    end
end

In [25]:
#optimal proposal
function propose(t, hkt, lkt, lmt, Pkt_array)
    #it just searches for the corresponding Pkt
    
    (payoff_max, policy) = @views Pkt_array[t, hkt, lmt, end-1:end]
    
    return payoff_max, policy
end

#test with hk = 0.5 (index 50)
t, hkt, lkt, lmt, Pkt_array = T-1, 50, 1, 60, Pst_array

@btime propose(t, hkt, lkt, lmt, Pkt_array)

  36.724 ns (1 allocation: 32 bytes)


(0.041000000000000036, 1.0)

In [26]:
Pst_array[T-1, 50, 60,:]

6-element Array{Float64,1}:
 -1.0
 60.0
  0.026000000000000075
  0.026000000000000075
  0.041000000000000036
  1.0

In [29]:
s_L(T-1)

0.041000000000000036